# Deutsch-Jozsa Algorithm

Deutsch's problem, which was later generalized to the Deutsch-Josza algorithm, was the first quantum algorithm with an exponential speed-up compared to any classical algorithm. Though the problem itself hasn't found any usefulness, it also showed that a qauntum computer can answer a question way faster than a normal computer and with 100% probablity of getting the right answer (excluding noise). It later inspired Simon's algorithm which inspired Shor's algorithm.

#### Prerequisites:
* [Theory Basics: the Qubit](https://appliedqc.org/2020/01/20/Theory-Basics-The-Qubit.html)
* Dirac notation

In [1]:
# This may take a few seconds
import numpy as np
import pandas as pd
from qiskit import *
import matplotlib.pyplot as plt

## Deutsch Problem (and algorithm)
We'll first start with Deutsch's original algorithm. The problem it answered goes like this. Given an unknown (aka black box) binary function $f(x)$ that maps one bit to another bit, $f(x):\{0,1\} \rightarrow\{0,1\}$, determine if $f(x)$ is constant or balanced. A constant function means $f(0)=f(1)$ while a balanced function means $f(0)\neq f(1)$ (since we're only considering binary input and output). For one input qubit and one output qubit, there are only four possible binary functions. Let's name them $f_0,f_1,f_2, \text{and} f_3$ and tabulate their possible outputs.

$$
\begin{array}{c|c|c}
  & x=0 & x=1 \\\hline
  f_0(x)= & 0 & 0 \\\hline
  f_1(x)= & 0 & 1 \\\hline
  f_2(x)= & 1 & 0 \\\hline
  f_3(x)= & 1 & 1 \\
\end{array}
$$

From this table, we see $f_0$ and $f_3$ are constant functions while $f_1$ and $f_2$ are balanced functions. Now, let's say a friend randomly picks one of these functions but hides which one by calling it $f(x)$. Our friend implements the randomly chosen $f(x)$ as a unitary (probability conserving) operation $U_f$ that spans the input (q_0) and output (q_1) qubits like so...

In [67]:
circuit = QuantumCircuit(2,1)
# circuit.h([0,1])
# circuit.barrier()
circuit.append(qiskit.circuit.Gate(name='U_f', num_qubits=2, params=[]), [0,1])
# circuit.barrier()
# circuit.h([0,1])
# circuit.measure([0, 1], [0, 1])
# Visualize the constructed circuit
circuit.draw()

The classical approach would be to try every possible input and measure the output. Once we tried both 0 and 1 as inputs, we'd know if $f(x)$ is balanced or not.

The quantum approach (1) makes a superposition of all possible inputs and (2) interferes the possibilities such that a 0 measured on q_0 (the input qubit) means $f(x)$ is constant and 1 measured on q_0 means $f(x)$ is balanced.

Let's do it!

To makes a superposition of all possible inputs, we apply a Hadamard (H) to the input qubit q_0. To get the right interference q_1 needs to be a 1 and also in a superposition, so it will get gates X then H. We apply the mystery function as a unitary operation on the qubits $U_f$. To complete the interference, every qubit gets and H after $U_f$. Then we measure. Our circuit will look something like this.

In [69]:
circuit = QuantumCircuit(2,2)
circuit.x(1)
circuit.h([0,1])
circuit.barrier()
circuit.append(qiskit.circuit.Gate(name='U_f', num_qubits=2, params=[]), [0,1])
circuit.barrier()
circuit.h([0,1])
circuit.measure([0, 1], [0, 1])
# Visualize the constructed circuit
circuit.draw()

For sake of experimenting, let's pick some $U_f$s and see if the algorithm works!

In [71]:
circuit = QuantumCircuit(2,2)
circuit.x(1)
circuit.h([0,1])
circuit.barrier()
circuit.id([0,1])
circuit.barrier()
circuit.h([0,1])
circuit.measure([0, 1], [0, 1])
# Visualize the constructed circuit
circuit.draw()

AttributeError: 'QuantumCircuit' object has no attribute 'id'

## Deutsch-Jozsa Problem (and algorithm)